## Install the dependencies and Import them

In [57]:

import torch
import torch.nn as nn

import lightning.pytorch as pl
pl.seed_everything(hash("setting random seeds") % 2**32 - 1)

from lightning.pytorch.loggers import WandbLogger

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
from torchvision import transforms
import random
from torchmetrics import JaccardIndex
import albumentations as A
from segmentation_models_pytorch.losses import DiceLoss
from torch.utils.data import DataLoader, Dataset
import glob

#from torch.optim.lr_scheduler import StepLR
#import torch.optim.lr_scheduler as lr_scheduler
from PIL import Image
import wandb

Seed set to 3494787661


In [58]:
torch.set_float32_matmul_precision('medium' )

## WANDB SECTION

In [59]:
wandb.login()

True

## Wandb sweep setup

In [60]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'acc',
    'goal': 'maximize'
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'batch_size':{
        'values':[5,10,15,20]
    },
    'learning_rate': {
    'distribution': 'log_uniform_values',
    'min': 5e-6,
    'max': 5e-2
    },
    'image_ip_size':{
        'values': [224,512]
    }
    }

parameters_dict.update({
    'epochs':{
        'value': 5
    }
})

sweep_config['parameters'] = parameters_dict

In [61]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'acc'},
 'parameters': {'batch_size': {'values': [5, 10, 15, 20]},
                'epochs': {'value': 5},
                'image_ip_size': {'values': [224, 512]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.05,
                                  'min': 5e-06},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [62]:
sweep_id = wandb.sweep(sweep_config, project ='try3')

Create sweep with ID: a6tls5ai
Sweep URL: https://wandb.ai/dayaalex/try3/sweeps/a6tls5ai


## DATASET SECTION 

## Augmentation functions

In [63]:
def aug_train(img_size):
    return A.Compose([
                        A.Resize(img_size,img_size),
                        A.HorizontalFlip(p= 0.5)
                    ])
    
def aug_val(img_size): 
    return A.Resize(img_size, img_size)
   
def aug_test(img_size):
    return A.Resize(img_size, img_size)

def norm_transform():
    return transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.3576, 0.3713, 0.3657], std=[0.2608, 0.2723, 0.2943])
        ])


## Normalisation

In [64]:
def verify_dataset_normalization(loader):
    # Initialize accumulators
    channel_sum = torch.tensor([0.0, 0.0, 0.0])
    channel_squared_sum = torch.tensor([0.0, 0.0, 0.0])
    num_batches = 0

    for images, _ in loader:
        # Accumulate sum and squared sum for each channel
        channel_sum += images.sum(dim=[0, 2, 3])
        channel_squared_sum += (images ** 2).sum(dim=[0, 2, 3])
        num_batches += images.shape[0]

    # Calculate mean and standard deviation
    mean = channel_sum / (num_batches *224  *224 )
    std = (channel_squared_sum / (num_batches * 224 *224 ) - mean ** 2) ** 0.5

    return mean, std

## Create custom dataset, dataloader

In [65]:
class CustomDataset(Dataset):
    def __init__(self, training_path, segmented_path,road_idx,norm_transform, augment):
        #"C:\Users\dalex\Desktop\Daya\datasets\idd-lite\idd20k_lite\leftImg8bit\train\2\593144_image.jpg"
        ##print('paths',training_path,segmented_path)
        
        self.filenames_t = glob.glob(training_path+f'*\*.jpg')
        ##print(self.filenames_t)
        self.filenames_s = glob.glob(segmented_path+f'*\*[0-9]_label.png')
        ##print(self.filenames_s)
        self.norm_transform = norm_transform
        self.augment = augment
        self.road_idx = road_idx

    def __len__(self):
        return len(self.filenames_t)

    def __getitem__(self, idx):
        #print('inside dataset')
        img_path = self.filenames_t[idx]
        mask_path = self.filenames_s[idx]
        img = Image.open(img_path).convert('RGB')# as PIL is used image is read as channel, height, width
        label_array = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)# image is read as height, width

        #extracting mask of road using road_idx
        road_mask = np.zeros_like(label_array)
        road_mask[label_array == self.road_idx] = 1
        road_mask = np.expand_dims(road_mask,axis = -1)
        
        if self.augment:
            img, mask = self.augment(img, road_mask)

        if self.norm_transform:
            img = self.norm_transform(img)
            #print('normalisation done')
            
        if not isinstance(mask, torch.Tensor):
            mask = torch.tensor(mask, dtype=torch.uint8)
            #print('mask turned to tensor')
            
        mask = mask.permute(2, 0, 1)
        #print('batched input and labels')
        return img, mask


## Train and validation dataloaders

In [66]:
class idd_lite_datamodule(pl.LightningDataModule):

    def __init__(self,image_ip_size,batch_size):
        super().__init__()
        self.image_ip_size = image_ip_size
        self.batch_size = batch_size
        
        
    def setup(self,stage=None):
        self.train_dataset = CustomDataset("c:\\Users\\dalex\\Desktop\\Daya\\datasets\\idd-lite\\idd20k_lite\\leftImg8bit\\train\\",
                                    "c:\\Users\\dalex\\Desktop\\Daya\\datasets\\idd-lite\\idd20k_lite\\gtFine\\train\\", 
                                    road_idx=0,
                                    norm_transform= norm_transform(),
                                    augment=aug_train(self.image_ip_size))


        self.val_dataset = CustomDataset("c:\\Users\\dalex\\Desktop\\Daya\\datasets\\idd-lite\\idd20k_lite\\leftImg8bit\\val\\",
                                    "c:\\Users\\dalex\\Desktop\\Daya\\datasets\\idd-lite\\idd20k_lite\\gtFine\\val\\", 
                                    road_idx=0,
                                    norm_transform= norm_transform(),
                                    augment=aug_val(self.image_ip_size))
        
        
        #print('preprocesiing setup')

    def train_dataloader(self):
        #print('inside train dataloader')
        # Instantiate your dataset
        
        tr_data_loader  = DataLoader(self.train_dataset, batch_size= self.batch_size, shuffle=True)
        
        return tr_data_loader

    def val_dataloader(self):
    
        # Instantiate your dataset
        
        val_data_loader = DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False) 

        return val_data_loader



## DEVICE

In [67]:
# Checking if there is any gpu available and pass the model to gpu or cpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## MODEL DEFINITIONS

trying out variations between joined training and seperated training

In [68]:
class InitialBlock(nn.Module):
  
  # Initial block of the model:
  #         Input
  #        /     \
  #       /       \
  #maxpool2d    conv2d-3x3
  #       \       /  
  #        \     /
  #      concatenate
   
    def __init__ (self,in_channels = 3,out_channels = 13):
        super().__init__()


        self.maxpool = nn.MaxPool2d(kernel_size=2, 
                                      stride = 2, 
                                      padding = 0)

        self.conv = nn.Conv2d(in_channels, 
                                out_channels,
                                kernel_size = 3,
                                stride = 2, 
                                padding = 1)

        self.prelu = nn.PReLU(16)

        self.batchnorm = nn.BatchNorm2d(out_channels)
  
    def forward(self, x):
        
        main = self.conv(x)
        main = self.batchnorm(main)
        
        side = self.maxpool(x)
        
        # concatenating on the channels axis
        x = torch.cat((main, side), dim=1)
        x = self.prelu(x)
        
        return x

In [69]:
class UBNeck(nn.Module):
    
  # Upsampling bottleneck:
  #     Bottleneck Input
  #        /        \
  #       /          \
  # conv2d-1x1     convTrans2d-1x1
  #      |             | PReLU
  #      |         convTrans2d-3x3
  #      |             | PReLU
  #      |         convTrans2d-1x1
  #      |             |
  # maxunpool2d    Regularizer
  #       \           /  
  #        \         /
  #      Summing + PReLU
  #
  #  Params: 
  #  projection_ratio - ratio between input and output channels
  #  relu - if True: relu used as the activation function else: Prelu us used
  
    def __init__(self, in_channels, out_channels, relu=False, projection_ratio=4):
        
        super().__init__()
        
        # Define class variables
        self.in_channels = in_channels
        self.reduced_depth = int(in_channels / projection_ratio)
        self.out_channels = out_channels
        
        
        if relu:
            activation = nn.ReLU()
        else:
            activation = nn.PReLU()
        
        self.unpool = nn.MaxUnpool2d(kernel_size = 2,
                                     stride = 2)
        
        self.main_conv = nn.Conv2d(in_channels = self.in_channels,
                                    out_channels = self.out_channels,
                                    kernel_size = 1)
        
        self.dropout = nn.Dropout2d(p=0.1)
        
        
        self.convt1 = nn.ConvTranspose2d(in_channels = self.in_channels,
                               out_channels = self.reduced_depth,
                               kernel_size = 1,
                               padding = 0,
                               bias = False)
        
        
        self.prelu1 = activation
        
        # This layer used for Upsampling
        self.convt2 = nn.ConvTranspose2d(in_channels = self.reduced_depth,
                                  out_channels = self.reduced_depth,
                                  kernel_size = 3,
                                  stride = 2,
                                  padding = 1,
                                  output_padding = 1,
                                  bias = False)
        
        self.prelu2 = activation
        
        self.convt3 = nn.ConvTranspose2d(in_channels = self.reduced_depth,
                                  out_channels = self.out_channels,
                                  kernel_size = 1,
                                  padding = 0,
                                  bias = False)
        
        self.prelu3 = activation
        
        self.batchnorm = nn.BatchNorm2d(self.reduced_depth)
        self.batchnorm2 = nn.BatchNorm2d(self.out_channels)
        
    def forward(self, x, indices):
        x_copy = x
        
        # Side Branch
        x = self.convt1(x)
        x = self.batchnorm(x)
        x = self.prelu1(x)
        
        x = self.convt2(x)
        x = self.batchnorm(x)
        x = self.prelu2(x)
        
        x = self.convt3(x)
        x = self.batchnorm2(x)
        
        x = self.dropout(x)
        
        # Main Branch
        
        x_copy = self.main_conv(x_copy)
        x_copy = self.unpool(x_copy, indices, output_size=x.size())
        
        # summing the main and side branches
        x = x + x_copy
        x = self.prelu3(x)
        
        return x

In [70]:
class RDDNeck(nn.Module):
    def __init__(self, dilation, in_channels, out_channels, down_flag, relu=False, projection_ratio=4, p=0.1):
      
  # Regular|Dilated|Downsampling bottlenecks:
  #
  #     Bottleneck Input
  #        /        \
  #       /          \
  # maxpooling2d   conv2d-1x1
  #      |             | PReLU
  #      |         conv2d-3x3
  #      |             | PReLU
  #      |         conv2d-1x1
  #      |             |
  #  Padding2d     Regularizer
  #       \           /  
  #        \         /
  #      Summing + PReLU
  #
  # Params: 
  #  dilation (bool) - if True: creating dilation bottleneck
  #  down_flag (bool) - if True: creating downsampling bottleneck
  #  projection_ratio - ratio between input and output channels
  #  relu - if True: relu used as the activation function else: Prelu us used
  #  p - dropout ratio
        
        super().__init__()
        
        # Define class variables
        self.in_channels = in_channels
        
        self.out_channels = out_channels
        self.dilation = dilation
        self.down_flag = down_flag
        
        # calculating the number of reduced channels
        if down_flag:
            self.stride = 2
            self.reduced_depth = int(in_channels // projection_ratio)
        else:
            self.stride = 1
            self.reduced_depth = int(out_channels // projection_ratio)
        
        if relu:
            activation = nn.ReLU()
        else:
            activation = nn.PReLU()
        
        self.maxpool = nn.MaxPool2d(kernel_size = 2,
                                      stride = 2,
                                      padding = 0, return_indices=True)
        

        
        self.dropout = nn.Dropout2d(p=p)

        self.conv1 = nn.Conv2d(in_channels = self.in_channels,
                               out_channels = self.reduced_depth,
                               kernel_size = 1,
                               stride = 1,
                               padding = 0,
                               bias = False,
                               dilation = 1)
        
        self.prelu1 = activation
        
        self.conv2 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.reduced_depth,
                                  kernel_size = 3,
                                  stride = self.stride,
                                  padding = self.dilation,
                                  bias = True,
                                  dilation = self.dilation)
                                  
        self.prelu2 = activation
        
        self.conv3 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.out_channels,
                                  kernel_size = 1,
                                  stride = 1,
                                  padding = 0,
                                  bias = False,
                                  dilation = 1)
        
        self.prelu3 = activation
        
        self.batchnorm = nn.BatchNorm2d(self.reduced_depth)
        self.batchnorm2 = nn.BatchNorm2d(self.out_channels)
        
        
    def forward(self, x):
        
        bs = x.size()[0]
        x_copy = x
        
        # Side Branch
        x = self.conv1(x)
        x = self.batchnorm(x)
        x = self.prelu1(x)
        
        x = self.conv2(x)
        x = self.batchnorm(x)
        x = self.prelu2(x)
        
        x = self.conv3(x)
        x = self.batchnorm2(x)
                
        x = self.dropout(x)
        
        # Main Branch
        if self.down_flag:
            x_copy, indices = self.maxpool(x_copy)
          
        if self.in_channels != self.out_channels:
            out_shape = self.out_channels - self.in_channels
            
            #padding and concatenating in order to match the channels axis of the side and main branches
            extras = torch.zeros((bs, out_shape, x.shape[2], x.shape[3]))
            extras = extras.to(device)
            x_copy = torch.cat((x_copy, extras), dim = 1)

        # Summing main and side branches
        x = x + x_copy
        x = self.prelu3(x)
        
        if self.down_flag:
            return x, indices
        else:
            return x

In [71]:
class ASNeck(nn.Module):
    def __init__(self, in_channels, out_channels, projection_ratio=4):
      
  # Asymetric bottleneck:
  #
  #     Bottleneck Input
  #        /        \
  #       /          \
  #      |         conv2d-1x1
  #      |             | PReLU
  #      |         conv2d-1x5
  #      |             |
  #      |         conv2d-5x1
  #      |             | PReLU
  #      |         conv2d-1x1
  #      |             |
  #  Padding2d     Regularizer
  #       \           /  
  #        \         /
  #      Summing + PReLU
  #
  # Params:    
  #  projection_ratio - ratio between input and output channels
        
        super().__init__()
        
        # Define class variables
        self.in_channels = in_channels
        self.reduced_depth = int(in_channels / projection_ratio)
        self.out_channels = out_channels
        
        self.dropout = nn.Dropout2d(p=0.1)
        
        self.conv1 = nn.Conv2d(in_channels = self.in_channels,
                               out_channels = self.reduced_depth,
                               kernel_size = 1,
                               stride = 1,
                               padding = 0,
                               bias = False)
        
        self.prelu1 = nn.PReLU()
        
        self.conv21 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.reduced_depth,
                                  kernel_size = (1, 5),
                                  stride = 1,
                                  padding = (0, 2),
                                  bias = False)
        
        self.conv22 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.reduced_depth,
                                  kernel_size = (5, 1),
                                  stride = 1,
                                  padding = (2, 0),
                                  bias = False)
        
        self.prelu2 = nn.PReLU()
        
        self.conv3 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.out_channels,
                                  kernel_size = 1,
                                  stride = 1,
                                  padding = 0,
                                  bias = False)
        
        self.prelu3 = nn.PReLU()
        
        self.batchnorm = nn.BatchNorm2d(self.reduced_depth)
        self.batchnorm2 = nn.BatchNorm2d(self.out_channels)
        
    def forward(self, x):
        bs = x.size()[0]
        x_copy = x
        
        # Side Branch
        x = self.conv1(x)
        x = self.batchnorm(x)
        x = self.prelu1(x)
        
        x = self.conv21(x)
        x = self.conv22(x)
        x = self.batchnorm(x)
        x = self.prelu2(x)
        
        x = self.conv3(x)
                
        x = self.dropout(x)
        x = self.batchnorm2(x)
        
        # Main Branch
        
        if self.in_channels != self.out_channels:
            out_shape = self.out_channels - self.in_channels
            
            #padding and concatenating in order to match the channels axis of the side and main branches
            extras = torch.zeros((bs, out_shape, x.shape[2], x.shape[3]))
            extras = extras.to(x_copy.device)
            x_copy = torch.cat((x_copy, extras), dim = 1)
        
        # Summing main and side branches
        x = x + x_copy
        x = self.prelu3(x)
        
        return x

In [72]:
class joined_ENet(pl.LightningModule):
  
  # Creating Enet model!
  
    def __init__(self, lr,optim):
        super().__init__()
        
        # Define class variables
        # C - number of classes
        self.C = 1
        self.lr = lr
        self.optimizer_name = optim
        self.save_hyperparameters()

        self.train_acc = JaccardIndex(task = 'multiclass', num_classes = 2)
        self.val_acc = JaccardIndex(task = 'multiclass', num_classes = 2)
        self.test_acc = JaccardIndex(task = 'multiclass', num_classes = 2)

        # The initial block
        self.init = InitialBlock()
        
        
        # The first bottleneck
        self.b10 = RDDNeck(dilation=1, 
                           in_channels=16, 
                           out_channels=64, 
                           down_flag=True, 
                           p=0.01)
        
        self.b11 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b12 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b13 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b14 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        
        # The second bottleneck
        self.b20 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=128, 
                           down_flag=True)
        
        self.b21 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b22 = RDDNeck(dilation=2, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b23 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b24 = RDDNeck(dilation=4, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b25 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b26 = RDDNeck(dilation=8, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b27 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b28 = RDDNeck(dilation=16, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        
        # The third bottleneck
        self.b31 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b32 = RDDNeck(dilation=2, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b33 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b34 = RDDNeck(dilation=4, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b35 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b36 = RDDNeck(dilation=8, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b37 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b38 = RDDNeck(dilation=16, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        
        # The fourth bottleneck
        self.b40 = UBNeck(in_channels=128, 
                          out_channels=64, 
                          relu=True)
        
        self.b41 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           relu=True)
        
        self.b42 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           relu=True)
        
        
        # The fifth bottleneck
        self.b50 = UBNeck(in_channels=64, 
                          out_channels=16, 
                          relu=True)
        
        self.b51 = RDDNeck(dilation=1, 
                           in_channels=16, 
                           out_channels=16, 
                           down_flag=False, 
                           relu=True)
        
        
        # Final ConvTranspose Layer
        self.fullconv = nn.ConvTranspose2d(in_channels=16, 
                                           out_channels=self.C, 
                                           kernel_size=3, 
                                           stride=2, 
                                           padding=1, 
                                           output_padding=1,
                                           bias=False)
        
        
    def forward(self, x):
        
        # The initial block
        x = self.init(x)
        
        # The first bottleneck
        x, i1 = self.b10(x)
        x = self.b11(x)
        x = self.b12(x)
        x = self.b13(x)
        x = self.b14(x)
        
        # The second bottleneck
        x, i2 = self.b20(x)
        x = self.b21(x)
        x = self.b22(x)
        x = self.b23(x)
        x = self.b24(x)
        x = self.b25(x)
        x = self.b26(x)
        x = self.b27(x)
        x = self.b28(x)
        
        # The third bottleneck
        x = self.b31(x)
        x = self.b32(x)
        x = self.b33(x)
        x = self.b34(x)
        x = self.b35(x)
        x = self.b36(x)
        x = self.b37(x)
        x = self.b38(x)
        
        # The fourth bottleneck
        x = self.b40(x, i2)
        x = self.b41(x)
        x = self.b42(x)
        
        # The fifth bottleneck
        x = self.b50(x, i1)
        x = self.b51(x)
        
        # Final ConvTranspose Layer
        x = self.fullconv(x)
        #print('finished parsing through model')

        return x
    
    
    def loss(self, inputs, target):
        if target.dtype == torch.uint8:
            target = target.float()

        #print(inputs.shape)
        #print(target.shape)
        pred = self(inputs.float()) 
        
        loss1 = DiceLoss(mode='binary')(pred, target.float())
        loss2 = nn.BCEWithLogitsLoss()(pred, target.float())

        #print((loss1+loss2).shape)
        return (loss1 + loss2), pred   

In [73]:
class encoder(nn.Module):
    def __init__(self, C):
        super().__init__()
        
        # Define class variables
        # C - number of classes
        self.C = C
        
        # The initial block
        self.init_layer = InitialBlock()
        
        
        # The first bottleneck
        self.b10 = RDDNeck(dilation=1, 
                           in_channels=16, 
                           out_channels=64, 
                           down_flag=True, 
                           p=0.01)
        
        self.b11 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b12 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b13 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b14 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        
        # The second bottleneck
        self.b20 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=128, 
                           down_flag=True)
        
        self.b21 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b22 = RDDNeck(dilation=2, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b23 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b24 = RDDNeck(dilation=4, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b25 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b26 = RDDNeck(dilation=8, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b27 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b28 = RDDNeck(dilation=16, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        
        # The third bottleneck
        self.b31 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b32 = RDDNeck(dilation=2, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b33 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b34 = RDDNeck(dilation=4, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b35 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b36 = RDDNeck(dilation=8, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b37 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b38 = RDDNeck(dilation=16, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
    def forward(self, x):
        
        # The initial block
        x = self.init_layer(x)
        
        # The first bottleneck
        x, i1 = self.b10(x)
        x = self.b11(x)
        x = self.b12(x)
        x = self.b13(x)
        x = self.b14(x)
        
        # The second bottleneck
        x, i2 = self.b20(x)
        x = self.b21(x)
        x = self.b22(x)
        x = self.b23(x)
        x = self.b24(x)
        x = self.b25(x)
        x = self.b26(x)
        x = self.b27(x)
        x = self.b28(x)
        
        # The third bottleneck
        x = self.b31(x)
        x = self.b32(x)
        x = self.b33(x)
        x = self.b34(x)
        x = self.b35(x)
        x = self.b36(x)
        x = self.b37(x)
        x = self.b38(x)

        return x, i1,i2

In [74]:
class decoder(nn.Module):
    def __init__(self, C):
        super().__init__()
        
        # Define class variables
        # C - number of classes
        self.C = C


        # The fourth bottleneck
        self.b40 = UBNeck(in_channels=128, 
                          out_channels=64, 
                          relu=True)
        
        self.b41 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           relu=True)
        
        self.b42 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           relu=True)
        
        
        # The fifth bottleneck
        self.b50 = UBNeck(in_channels=64, 
                          out_channels=16, 
                          relu=True)
        
        self.b51 = RDDNeck(dilation=1, 
                           in_channels=16, 
                           out_channels=16, 
                           down_flag=False, 
                           relu=True)
        
        
        # Final ConvTranspose Layer
        self.fullconv = nn.ConvTranspose2d(in_channels=16, 
                                           out_channels=self.C, 
                                           kernel_size=3, 
                                           stride=2, 
                                           padding=1, 
                                           output_padding=1,
                                           bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x, i1, i2):
        # The fourth bottleneck
        x = self.b40(x, i2)
        x = self.b41(x)
        x = self.b42(x)
        
        # The fifth bottleneck
        x = self.b50(x, i1)
        x = self.b51(x)
        
        # Final ConvTranspose Layer
        x = self.fullconv(x)
        x = self.sigmoid(x)
        return x

In [75]:
#ENCODER-DECODER
class enc_dec_ENet(pl.LightningModule):
  
  # Creating Enet model!
  
    def __init__(self,encoder,decoder, C):
        super().__init__()
        
        self.encoder = encoder(C)
        
        # Create the decoder
        self.decoder = decoder(C)

        self.save_hyperparameters()

        self.train_acc = JaccardIndex(task = 'multiclass', num_classes = 2)
        self.val_acc = JaccardIndex(task = 'multiclass', num_classes = 2)

    def forward(self, x):
        # Pass input through the encoder
        encoder_output, i1, i2 = self.encoder(x)

        # Pass encoder output and intermediate feature maps to the decoder
        decoder_output = self.decoder(encoder_output, i1, i2)
        #print('finished parsing through the model')
        return decoder_output

    def combined_loss(self, inputs, target):
        #print('inside loss function')
        if target.dtype == torch.uint8:
            target = target.float()

        pred = self(inputs.float()) 
        #print('predictions found')
        loss1 = DiceLoss(mode='binary')(pred, target)
        loss2 = nn.BCEWithLogitsLoss()(pred, target)
        return (loss1 + loss2), pred
  

In [76]:
def configure_optimizers(self):
    #print('inside optimiser function')
    if self.optimizer_name =='adam':
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=2e-4)
    elif self.optimizer_name == 'sgd':
        optimizer = torch.optim.SGD(self.parameters(),
                                lr=self.lr, momentum=0.9)
    else:
        raise ValueError(f"Unsupported optimizer: {self.optimizer_name}")
    #print('optimizer ready')
    return optimizer

## Training loop

In [77]:

# Train loop
def training_step(self,batch, batch_idx):
    #print('inside train function')
    X_batch, mask_batch = batch
    #print('batch selected')
    net_loss, preds = self.loss(X_batch, mask_batch.float())
    #print('loss computed')
    preds_ohe = torch.argmax(preds, dim=1)
    mask_batch_1d = mask_batch.squeeze(1)
    
    self.train_acc(preds_ohe,mask_batch_1d)
    #print('accuracy computed')
    self.log('train/loss', net_loss, on_epoch = True )
    self.log('train/acc', self.train_acc, on_epoch = True, logger=True)

    return net_loss#loss must be returned to facilitate grad calculation


In [78]:
def test_step(self, batch, batch_idx):
    X_batch, mask_batch = batch
    net_loss, preds = self.loss(X_batch, mask_batch.float())
    preds_ohe =torch.argmax(preds, dim = 1)
    mask_batch_1d = mask_batch.squeeze(1)
    self.test_acc(preds_ohe, mask_batch_1d)

    self.log('test/loss', net_loss,on_step = False, on_epoch =True)
    self.log('test/acc', self.test_acc, on_step = False, on_epoch = True)

In [79]:
def on_test_epoch_end(self):
    dummy_input = torch.zeros_like(self.img_aug_func_train, device = self.device)
    model_filename = 'enet.onnx'
    self.to_onnx(model_filename, dummy_input, export_params= True)
    artifact = wandb.Artifact('model.ckpt', type = 'model')
    artifact.add_file(model_filename)
    wandb.log_artifact(artifact)
    

In [80]:
#logging histogram of logits during validation
def on_validation_epoch_start(self):
    self.val_steps_output = []#to store all the logits

def validation_step(self, batch, batch_idx):
    X_batch, mask_batch = batch
    net_loss, preds = self.loss(X_batch, mask_batch.float())
    preds_ohe =torch.argmax(preds, dim = 1)
    mask_batch_1d = mask_batch.squeeze(1)
    self.val_acc(preds_ohe, mask_batch_1d)

    self.log('val/loss_epoch', net_loss,on_step = False, on_epoch =True)
    self.log('val/acc_loss_epoch', self.test_acc, on_step = False, on_epoch = True)
    self.val_steps_output.append(preds)
    return preds

def on_validation_epoch_end(self):
    validation_step_outputs = self.val_steps_output

    dummy_input = torch.zeros_like(self.img_aug_func_train, device = self.device)
    model_filename = f"enet_model_{str(self.global_step).zfill(5)}.onnx"
    torch.onnx.export(self, dummy_input, model_filename, opset_version=11)
    artifact = wandb.Artifact(name="model.ckpt", type="model")
    artifact.add_file(model_filename)
    self.logger.experiment.log_artifact(artifact)

    flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
    self.logger.experiment.log(
        {"valid/logits": wandb.Histogram(flattened_logits.to("cpu")),
         "global_step": self.global_step})

## Callback with image viewing

In [81]:
class ImagePredictionLogger(pl.Callback):
    def __init__(self, val_samples, num_samples=10):
        super().__init__()
        self.inputs, self.gt = val_samples
        self.inputs = self.inputs[:num_samples]
        self.gt = self.gt[:num_samples]

    def on_validation_epoch_end(self, trainer, pl_module):
        self.inputs = self.inputs.to(pl_module.device)
        pred = pl_module(self.inputs)
        pred_segmap = torch.argmax(pred,1)
        
        data = []

        for x, y, z in zip(self.inputs, pred_segmap, self.gt):
            x=x.squeeze(0).cpu().detach().numpy()
            x = x.transpose(1,2,0)
            y=y.squeeze(0).cpu().detach().numpy()
            colormap = plt.get_cmap('viridis')  # Or any other colormap
            y_colored = colormap(y)[:, :, :3]   
            
            z=z.squeeze(0).cpu().detach().numpy()
            z_colored = colormap(y)[:,:,:3]

            logged_x = wandb.Image(x) 
            logged_y = wandb.Image(y_colored)
            logged_z = wandb.Image(z_colored)

            data.append([logged_x,logged_y,logged_z])

        trainer.logger.experiment.log({
            'val images table':wandb.Table(data = data, columns =['Inputs', 'Predictions', 'Ground Truth'])

        })

In [85]:
def train_using_wandb(config = None):
    with wandb.init(config = config):
        config = wandb.config# automatically set by sweepcontroller previously configuired
        logger = WandbLogger(project='try3')
        
        idd_data = idd_lite_datamodule(config.image_ip_size,config.batch_size)
        
        idd_data.setup()
        joined_ENet.training_step = training_step
        joined_ENet.configure_optimizers = configure_optimizers
        joined_ENet.on_validation_epoch_start = on_validation_epoch_start
        joined_ENet.validation_step = validation_step
        joined_ENet.on_validation_epoch_end = on_validation_epoch_end

        
        model = joined_ENet(config.learning_rate, config.optimizer)        
        wandb.watch(model,model.loss, log='all', log_freq= 10)
        samples = next(iter(idd_data.val_dataloader()))


        trainer = pl.Trainer(
        logger = logger,
        log_every_n_steps = 50,
        max_epochs = config.epochs,
        deterministic = False,
        callbacks = [ImagePredictionLogger(samples)]
        )

        trainer.fit(model, datamodule=idd_data)
        wandb.finish()

In [86]:
wandb.agent(sweep_id, train_using_wandb, count=5)

wandb: Agent Starting Run: x5ed34m2 with config:
wandb: 	batch_size: 5
wandb: 	epochs: 5
wandb: 	image_ip_size: 512
wandb: 	learning_rate: 0.039570078615655264
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\loggers\wandb.py:390: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type                   | Params
------------------------------------------------------
0  | train_acc | MulticlassJaccardIndex | 0     
1  | val_acc   | MulticlassJaccardIndex | 0     
2  | test_acc  | MulticlassJaccardIndex | 0     
3  | init      | InitialBlock           | 406   
4  | b10       | RDDNeck                | 605   
5  | b11       | RDDNeck                | 4.5 K 
6  | b12       | RDDNeck                | 4.5 K 
7  | b13       | RDDNeck     

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  6.74it/s]

...

Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  5.16it/s]

c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  9.12it/s]

Traceback (most recent call last):
  File "C:\Users\dalex\AppData\Local\Temp\ipykernel_33700\2073484368.py", line 30, in train_using_wandb
    trainer.fit(model, datamodule=idd_data)
  File "c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\trainer.py", line 543, in fit
    call._call_and_handle_interrupt(
  File "c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\call.py", line 44, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\trainer.py", line 579, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\trainer.p

Run pnz4n8de errored:
Traceback (most recent call last):
  File "c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\wandb\agents\pyagent.py", line 308, in _run_job
    self._function()
  File "C:\Users\dalex\AppData\Local\Temp\ipykernel_33700\2073484368.py", line 30, in train_using_wandb
    trainer.fit(model, datamodule=idd_data)
  File "c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\trainer.py", line 543, in fit
    call._call_and_handle_interrupt(
  File "c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\call.py", line 44, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dalex\Desktop\Daya\ENet-Real-Time-Semantic-Segmentation\daya_pytorch_env\Lib\site-packages\lightning\pytorch\trainer\trainer.py", line 579,

Define the function that maps a 2D image with all the class labels to a segmented image with the specified colored maps

In [84]:
def decode_segmap(image):
    Sky = [128, 128, 128]
    Building = [128, 0, 0]
    Pole = [192, 192, 128]
    Road_marking = [255, 69, 0]
    Road = [128, 64, 128]
    Pavement = [60, 40, 222]
    Tree = [128, 128, 0]
    SignSymbol = [192, 128, 128]
    Fence = [64, 64, 128]
    Car = [64, 0, 128]
    Pedestrian = [64, 64, 0]
    Bicyclist = [0, 128, 192]

    label_colours = np.array([Sky, Building, Pole, Road_marking, Road, 
                              Pavement, Tree, SignSymbol, Fence, Car, 
                              Pedestrian, Bicyclist]).astype(np.uint8)
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    for l in range(0, 12):
        r[image == l] = label_colours[l, 0]
        g[image == l] = label_colours[l, 1]
        b[image == l] = label_colours[l, 2]

    rgb = np.zeros((image.shape[0], image.shape[1], 3)).astype(np.uint8)
    rgb[:, :, 0] = b
    rgb[:, :, 1] = g
    rgb[:, :, 2] = r
    return rgb